# COMP9033 - Data Analytics Lab 07a: $k$ nearest neighbours classification
## Introduction

This lab focuses on SMS message spam detection using $k$ nearest neighbours classification. It's a direct counterpart to the rule-based spam detection from Lab 05. At the end of the lab, you should be able to use `scikit-learn` to:

- Create a $k$ nearest neighbours classification model.
- Use the model to predict new values.
- Measure the accuracy of the model.

### Getting started

Let's start by importing the packages we'll need. This week, we're going to use the `neighbors` subpackage from `scikit-learn` to build k nearest neighbours models. We'll also use the `dummy` package to build a baseline model from we which can gauge how good our final model is.

In [ ]:
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction import stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

Next, let's load the data. Write the path to your `sms.csv` file in the cell below:

In [ ]:
data_file = 'data/sms.csv'

Execute the cell below to load the CSV data into a pandas data frame with the columns `label` and `message`.

> **Note:** This week, the CSV file is not comma separated, but instead tab separated. We can tell `pandas` about the different format using the `sep` argument, as shown in the cell below. For more information, see the `read_csv` [documentation](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html).

In [ ]:
sms = pd.read_csv(data_file, sep='\t', header=None, names=['label', 'message'])
sms.head()

## Data modelling

Before we start modelling, let's split our data into training and test so we can measure the accuracy of our final model:

In [ ]:
X = sms['message']
y = sms['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

### Data transformation

Our data is in the form of raw text. This was fine when we were using a rule-based model, but it won't work with $k$ nearest neighbours. Instead, we'll need to transform the data into a numerical representation. One popular way to do this with text data is to compute *term frequency* (TF) and *inverse document frequency* (IDF) measures:

- Term frequency is a measure of how often a given term appears in a given document, e.g. how often the word "free" appears in a given SMS message. The more often a word appears in a document, the higher its term frequency.
- Inverse document frequency is a measure of how rare a word is in a set of documents, e.g. the word "the" appears commonly in many SMS messages and so its presence (or absence) provides little information when it comes to distinguishing spam from ham. The higher the inverse document frequency of a word, the rarer it is (and, therefore, the more distinguishing power it has).

Typically, term frequency and inverse document frequency are combined as a single metric, [*term frequency-inverse document frequency*](https://en.wikipedia.org/wiki/Tf%E2%80%93idf) (TF-IDF), which is simply the multiple of the individual values. Consequently, if a term has a high TF-IDF score, its presence across a set of documents (e.g. SMS messages) is low, while its number of occurrences in a given document (e.g. a candidate SMS message under evaluation) is high. If a term has a low TF-IDF score, this is an indicator that it doesn't appear very frequently in a given document, occurs very frequently across the set of documents, or both. We can exploit this information to find terms that can distinguish a certain set of documents (e.g. spam) from a larger set of documents.


### Dummy modelling

Before we build the $k$ nearest neighbours model, let's build a dummy model, i.e. a naive model that predicts new values using a simple strategy. Dummy models are usually not good predictors (we usually won't use them to solve real problems), but are useful because they provide a baseline accuracy measurement for the data set, from which we can gauge the accuracy of any further models we build. In Lab 05, we built a rule-based model for SMS message spam detection for this purpose.

`scikit-learn` provides dummy model functionality via the [`dummy`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.dummy) subpackage. This subpackage provides both dummy regression and classification algorithms, which can be customised with different strategies. We can use the [`DummyClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.dummy.DummyClassifier.html#sklearn.dummy.DummyClassifier) class to build a dummy classification model. `DummyClassifier` supports five different strategies for predicting values:

1. `strategy='stratified'`: Predict new values randomly, but in proportion to their frequency in the training set.
2. `strategy='most_frequent'`: Predict new values as the most frequently occurring target variable in the training set.
3. `strategy='prior'`: Predict new values as the most frequently occurring target variable in the training set. This is essentially the same as `strategy='most_frequent'`, but returns different values when the `predict_proba` method is called.
4. `strategy='uniform'`: Predict new values randomly, with equal probability.
5. `strategy='constant'`: Predict new values as some constant value (the constant value must also be specified using the `constant` keyword argument).

Let's build a model that predicts new values in proportion to their occurrence in the training set. As usual, we create an instance of the model building class and then use the `fit` method to fit it to the training data:

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS), # Remove stop words before computing TF-IDF
    DummyClassifier(strategy='stratified')
)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

As can be seen, the dummy model is a poor fit for the data and performs worse than our simple rule-based model from Lab 05. But it does give us a baseline error level from which we can improve. Let's build a $k$ nearest neighbours model and see what the difference is.

### $k$ nearest neighbours modelling

Let's use model selection via cross validation to select the optimal $k$ nearest neighbours model from a set of candidates:

> **Note:** The grid search could take a few minutes to complete, depending on the amount of processing power  you have available.

In [ ]:
pipeline = make_pipeline(
    TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS),
    KNeighborsClassifier()
)

parameters = {
    'kneighborsclassifier__n_neighbors': [1, 2, 3, 4, 5, 10, 15, 20],
    'kneighborsclassifier__weights': ['uniform', 'distance'],
    'kneighborsclassifier__metric': ['manhattan', 'euclidean']
}

gs = GridSearchCV(pipeline, parameters, cv=5, n_jobs=-1) # n_jobs=-1 uses all available CPUs for calculation
gs.fit(X_train, y_train) # Fit using the training set

We can check the parameters of the selected model using the `best_params_` attribute of the fitted grid search:

In [ ]:
gs.best_params_

Finally, we can use our model to predict the classes of our test set data and print a classification report to compare the results to the dummy model above and the rule-based model from Lab 05:

In [ ]:
y_pred = gs.predict(X_test)

print(classification_report(y_test, y_pred))

The model is much more accurate than both the dummy model above and the rule-based model from Lab 05. Specifically, we can say that:

- 92% of the messages we labelled as ham were actually ham (precision for ham = 0.92).
- 100% of the messages we labelled as spam were actually spam (precision for spam = 1.00).
- We labelled every actual ham as ham (recall for ham = 1.00).
- We labelled 47% of spam as spam (recall for spam = 0.47).

While no ham was misclassified as spam, we only managed to filter 47% of spam emails (approximately one in every two). Next week, we'll use decision tree classifiers to improve on this.